# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [94]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [95]:
document_tree = ET.parse('data/mondial_database_less.xml')
document_tree

In [96]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [97]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('*' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

*Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
*Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
*Macedonia:
Skopje, Kumanovo
*Serbia:
Beograd, Novi Sad, Niš
*Montenegro:
Podgorica
*Kosovo:
Prishtine
*Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

****
## 10 countries with the lowest infant mortality rates

In [98]:
import pandas as pd
document = ET.parse('data/mondial_database.xml')
df = pd.DataFrame(columns=('Name','Mortality'))
i=0
for element in document.iterfind('country'):
    if(element.find('infant_mortality')!= None):
        row={'Name':element.find('name').text ,'Mortality': element.find('infant_mortality').text}
        #print row['Name']
        row_s = pd.Series(row)
        row_s.name=i
        i=i+1
        df = df.append(row_s)
    
df    
df=df.sort_values('Mortality',ascending=True)
df.head(10)    
    
    

,Name,Mortality
36,Monaco,1.81
28,Romania,10.16
142,Fiji,10.2
63,Brunei,10.48
124,Grenada,10.5
221,Mauritius,10.59
116,Panama,10.7
227,Seychelles,10.77
94,United Arab Emirates,10.92
105,Barbados,10.93


## 10 cities with the largest population

In [99]:
import pandas as pd
document = ET.parse('data/mondial_database.xml')
df3 = pd.DataFrame(columns=('Country','CityId','City','Year','Population'))
k=0
for element in document.iterfind('country'):
    for elementCity in element.findall('city'):
        #print (elementCity.attrib['id'])
        #print (elementCity.find('name').text)
        for population in elementCity.findall('population'):
            #print (population.attrib['year'])
            #print (population.text)
            if(population.attrib['year']=='2011'):
                row={'Country':element.find('name').text ,'CityId':elementCity.attrib['id'],'City':elementCity.find('name').text,\
                'Year':population.attrib['year'],'Population':population.text}
                row_s = pd.Series(row)
                row_s.name=k
                k=k+1
                df3 = df3.append(row_s)
    for province in element.findall('province'):
        for elementCity in province.findall('city'):
            #print (elementCity.attrib['id'])
            #print (elementCity.find('name').text)
            for population in elementCity.findall('population'):
                #print (population.attrib['year'])
                #print (population.text)
                if(population.attrib['year']=='2011'):
                    row={'Country':element.find('name').text ,'CityId':elementCity.attrib['id'],'City':elementCity.find('name').text,\
                    'Year':population.attrib['year'],'Population':population.text}
                    row_s = pd.Series(row)
                    row_s.name=k
                    k=k+1
                    df3 = df3.append(row_s)  
                      
    
df3=df3.sort_values('Population',ascending=False)
df3.head(10)

,Country,CityId,City,Year,Population
165,Germany,cty-Germany-63,Cottbus,2011,99984
183,Germany,cty-Germany-79,Hildesheim,2011,99554
387,United Kingdom,cty-United-Kingdom-12,Wakefield,2011,99251
210,Germany,cty-Germany-73,Siegen,2011,99187
182,Germany,cty-Germany-70,Salzgitter,2011,98895
346,Romania,cty-Romania-Ramnicu-Valcea,Râmnicu Vâlcea,2011,98776
144,Czech Republic,cty-Czech-Usti-nad-Labem,Ústí nad Labem,2011,98596
385,United Kingdom,cty-United-Kingdom-84,Preston,2011,97886
377,United Kingdom,cty-United-Kingdom-37,Oldham,2011,96555
212,Germany,cty-Germany-80,Witten,2011,96382


## 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [100]:
import pandas as pd
document = ET.parse('data/mondial_database.xml')
df4 = pd.DataFrame(columns=('Country','EthnicGroup','Population'))
k=0
for element in document.iterfind('country'):
    varpopulation=0
    for population in element.findall('population'):
        if(population.attrib['year']=='2011'):
            varpopulation=int(population.text)
    for ethnicgroup in element.findall('ethnicgroup'):
        percent=float(ethnicgroup.attrib['percentage'])/100
        row={'Country':element.find('name').text ,'EthnicGroup':ethnicgroup.text,\
             'Population':int(varpopulation*percent)}
        row_s = pd.Series(row)
        row_s.name=k
        k=k+1
        df4 = df4.append(row_s)
        
#df4.head(10)
df4=df4.groupby('EthnicGroup',as_index=False).sum()
#df4.rein
df4=df4.sort_values('Population',ascending=False)
df4.head(10)
#df5=df4[df4.EthnicGroup=='Albanian']
#df5[('Population')].sum()#4805362

,EthnicGroup,Population
120,Indo-Aryan,871815583.0
77,Dravidian,302713744.0
2,African,166391980.0
42,Bengali,146776916.0
105,German,74278483.0
84,English,52820300.0
173,Mediterranean Nordic,46815916.0
206,Persian,38326331.0
208,Polish,38018418.0
184,Mongol,36325649.0


## name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [123]:
import pandas as pd
document = ET.parse('data/mondial_database.xml')
df4 = pd.DataFrame(columns=('Country','Code'))
k=0
for element in document.iterfind('country'):
    #print (element.attrib['car_code'])
    row={'Country':element.find('name').text ,'Code':element.attrib['car_code']}
    row_s = pd.Series(row)
    row_s.name=k
    k=k+1
    df4 = df4.append(row_s)
#df4.head()

dfRiver=pd.DataFrame(columns=('RiverId','Name','Code','Length'))
k=0
for element in document.iterfind('river'):
    #print (element.attrib['id'])
    if(element.find('length')!= None):
        row={'RiverId':element.attrib['id'] ,'Name':element.find('name').text,'Code':element.attrib['country'],\
        'Length':float(element.find('length').text)}
        row_s = pd.Series(row)
        row_s.name=k
        k=k+1
        dfRiver = dfRiver.append(row_s)

df5=pd.merge(dfRiver, df4, on='Code')
#df5=dfRiver.join(df4)
df5=df5.sort_values('Length',ascending=False)
df5.head(1)

,RiverId,Name,Code,Length,Country
75,river-Jangtse,Jangtse,CN,6380.0,China


In [128]:
dfLake=pd.DataFrame(columns=('LakeId','Name','Code','Area'))
k=0
for element in document.iterfind('lake'):
    #print (element.attrib['id'])
    if(element.find('area')!= None):
        row={'LakeId':element.attrib['id'] ,'Name':element.find('name').text,'Code':element.attrib['country'],\
        'Area':float(element.find('area').text)}
        row_s = pd.Series(row)
        row_s.name=k
        k=k+1
        dfLake = dfLake.append(row_s)
#dfLake
df5=pd.merge(dfLake, df4, on='Code')
#df5=dfRiver.join(df4)
df5=df5.sort_values('Area',ascending=False)
df5.head(1)

,LakeId,Name,Code,Area,Country
75,lake-Lake_Michigan,Lake Michigan,USA,57800.0,United States
68,lake-Great_Bear_Lake,Great Bear Lake,CDN,31792.0,Canada
35,lake-Baikalsee,Ozero Baikal,R,31492.0,Russia
69,lake-Great_Slave_Lake,Great Slave Lake,CDN,28568.0,Canada
71,lake-Lake_Winnipeg,Lake Winnipeg,CDN,24420.0,Canada


In [138]:
dfAirport=pd.DataFrame(columns=('IataCode','Name','Code','Elevation'))
k=0
for element in document.iterfind('airport'):
    if(element.find('elevation').text!= None):
        row={'IataCode':element.attrib['iatacode'] ,'Name':element.find('name').text,'Code':element.attrib['country'],\
        'Elevation':float(element.find('elevation').text)}
        row_s = pd.Series(row)
        row_s.name=k
        k=k+1
        dfAirport = dfAirport.append(row_s)
        
#dfAirport
df5=pd.merge(dfAirport, df4, on='Code')
#df5=dfRiver.join(df4)
df5=df5.sort_values('Elevation',ascending=False)
df5.head(1)

,IataCode,Name,Code,Elevation,Country
80,LPB,El Alto Intl,BOL,4063.0,Bolivia
212,LXA,Lhasa-Gonggar,CN,4005.0,China
230,YUS,Yushu Batang,CN,3963.0,China
787,JUL,Juliaca,PE,3827.0,Peru
789,CUZ,Teniente Alejandro Velasco Astete Intl,PE,3311.0,Peru
